# 2023

In [1]:
import re
from operator import mul
from queue import Queue
import numpy as np 
from collections import Counter, defaultdict
from math import prod
import sys
from typing import List
from time import perf_counter
from matplotlib.path import Path
from time import perf_counter
from itertools import combinations
from tqdm import tqdm
from pprint import pprint
from copy import deepcopy
import heapq
from utils import *
from cProfile import run
from functools import cache
from sympy import var, Eq, solve
import networkx
import random

sys.setrecursionlimit(999999)

In [48]:
def j11():
    filename = "Input/inputj11.txt"     
       
    def partie1():
        with open(filename) as fin:
            lines = fin.read().strip().split("\n")

        n, m = len(lines), len(lines[0])

        coords = []
        for i in range(n):
            for j in range(m):
                if lines[i][j] == "#":
                    coords.append((i, j))

        N = len(coords)

        empty_row = [all([lines[i][j] == "." for j in range(m)]) for i in range(n)]
        empty_col = [all([lines[i][j] == "." for i in range(n)]) for j in range(m)]


        def dist(a, b):
            i1, j1 = a
            i2, j2 = b

            i1, i2 = min(i1, i2), max(i1, i2)
            j1, j2 = min(j1, j2), max(j1, j2)

            ans = 0
            for i in range(i1, i2):
                ans += 1
                if empty_row[i]:
                    ans += 1
            for j in range(j1, j2):
                ans += 1
                if empty_col[j]:
                    ans += 1

            return ans


        ans = 0
        for idx1 in range(N):
            for idx2 in range(idx1+1, N):
                d = dist(coords[idx1], coords[idx2])
                ans += d

        print(f"Réponse jour 11 partie 1 : {ans}")
    
    def partie2():
        with open(filename) as fin:
            lines = fin.read().strip().split("\n")

        n, m = len(lines), len(lines[0])

        coords = []
        for i in range(n):
            for j in range(m):
                if lines[i][j] == "#":
                    coords.append((i, j))

        N = len(coords)

        empty_row = [all([lines[i][j] == "." for j in range(m)]) for i in range(n)]
        empty_col = [all([lines[i][j] == "." for i in range(n)]) for j in range(m)]


        def dist(a, b):
            i1, j1 = a
            i2, j2 = b

            i1, i2 = min(i1, i2), max(i1, i2)
            j1, j2 = min(j1, j2), max(j1, j2)

            ans = 0
            for i in range(i1, i2):
                ans += 1
                if empty_row[i]:
                    ans += 10**6 - 1
            for j in range(j1, j2):
                ans += 1
                if empty_col[j]:
                    ans += 10**6 - 1

            return ans


        ans = 0
        for idx1 in range(N):
            for idx2 in range(idx1+1, N):
                d = dist(coords[idx1], coords[idx2])
                ans += d

        print(f"Réponse jour 11 partie 2 : {ans}")
    
    partie1()
    partie2()
    
j11()

Réponse jour 11 partie 1 : 9509330
Réponse jour 11 partie 2 : 635832237682


In [56]:
def j12():
    filename = "Input/inputj12.txt"
        
    def partie1():
        with open(filename) as fin:
            lines = fin.read().strip().split("\n")

        ss = []
        target_runs = []
        for line in lines:
            parts = line.split(" ")
            ss.append(parts[0])
            target_runs.append(list(map(int, parts[1].split(","))))


        def valid(line, target_runs):
            n = len(line)
            runs = []

            i = 0
            while i < n:
                while i < n and not line[i]:
                    i += 1
                if i == n:
                    break
                j = i
                c = 0
                while j < n and line[j]:
                    j += 1
                    c += 1

                runs.append(c)
                i = j

            return runs == target_runs


        def ways(s, target_runs):
            line = []
            idxs = []
            for i, x in enumerate(s):
                if x == ".":
                    line.append(0)
                if x == "?":
                    line.append(-1)
                    idxs.append(i)
                if x == "#":
                    line.append(1)

            count = 0
            for mask in range(1 << len(idxs)):
                line_copy = line.copy()
                for i in range(len(idxs)):
                    if mask & (1 << i):
                        line_copy[idxs[i]] = 0
                    else:
                        line_copy[idxs[i]] = 1

                if valid(line_copy, target_runs):
                    count += 1

            return count


        ans = 0
        for s, target_run in tqdm(list(zip(ss, target_runs))):
            res = ways(s, target_run)
            ans += res
            
        print(f"Réponse jour 12 partie 1 : {ans}")
    
    def partie2():
        with open(filename) as fin:
            lines = fin.read().strip().split("\n")

        ss = []
        target_runs = []
        for line in lines:
            parts = line.split(" ")
            parts[0] = "?".join([parts[0]] * 5)
            parts[1] = ",".join([parts[1]] * 5)

            ss.append(parts[0])
            target_runs.append(list(map(int, parts[1].split(","))))


        def ways(s, target_runs):
            # This is what I get for not investing in DP (dyanmic programming)
            # 3D dp on (idx in string, idx in set, length of current run)
            # Question: How many ways?
            target_runs.append(0)
            max_run = max(target_runs)
            s += "."

            n = len(s)
            m = len(target_runs)
            dp = [[[None for _ in range(max_run+1)]
                for _ in range(m)] for _ in range(n)]

            for i in range(n):
                x = s[i]
                for j in range(m):
                    for k in range(target_runs[j]+1):
                        # Base case
                        if i == 0:
                            if j != 0:
                                dp[i][j][k] = 0
                                continue

                            if x == "#":
                                if k != 1:
                                    dp[i][j][k] = 0
                                    continue
                                dp[i][j][k] = 1
                                continue

                            if x == ".":
                                if k != 0:
                                    dp[i][j][k] = 0
                                    continue
                                dp[i][j][k] = 1
                                continue

                            if x == "?":
                                if k not in [0, 1]:
                                    dp[i][j][k] = 0
                                    continue
                                dp[i][j][k] = 1
                                continue

                        # Process answer if current char is .
                        if k != 0:
                            ans_dot = 0
                        elif j > 0:
                            assert k == 0
                            ans_dot = dp[i-1][j-1][target_runs[j-1]]
                            ans_dot += dp[i-1][j][0]
                        else:
                            # i>0, j=0, k=0.
                            # Only way to do this is if every ? is a .
                            ans_dot = 1 if s[:i].count("#") == 0 else 0

                        # Process answer if current char is #
                        if k == 0:
                            ans_pound = 0
                        else:
                            # Newest set
                            ans_pound = dp[i-1][j][k-1]

                        if x == ".":
                            dp[i][j][k] = ans_dot
                        elif x == "#":
                            dp[i][j][k] = ans_pound
                        else:
                            dp[i][j][k] = ans_dot + ans_pound

            ans = dp[n-1][-1][0]

            return ans


        ans = 0
        for s, target_run in tqdm(list(zip(ss, target_runs)), ncols=70):
            res = ways(s, target_run)
            ans += res
            
        print(f"Réponse jour 12 partie 1 : {ans}")
    
    partie1()
    partie2()
    
j12()

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:12<00:00, 81.90it/s]


Réponse jour 12 partie 1 : 7191


100%|████████████████████████████| 1000/1000 [00:01<00:00, 912.19it/s]

Réponse jour 12 partie 1 : 6512849198636


In [55]:
def j13():
    filename = "Input/inputj13.txt"

    def partie1():
        with open(filename) as fin:
            patterns = [lines.split("\n") for lines in fin.read().strip().split("\n\n")]


        def is_horiz(grid, i):
            n, m = len(grid), len(grid[0])
            # Vertical line of reflection
            for j in range(m):
                for k1 in range(n):
                    k2 = i*2+1 - k1
                    if not (0 <= k2 < n):
                        continue
                    if grid[k1][j] != grid[k2][j]:
                        return False

            return True


        def transpose(grid):
            return list(zip(*grid))


        def summary(grid):
            n, m = len(grid), len(grid[0])

            horiz = -1
            for i in range(n-1):
                if is_horiz(grid, i):
                    horiz = i
                    break

            vert = -1
            T = transpose(grid)
            for j in range(m-1):
                if is_horiz(T, j):
                    vert = j
                    break

            assert vert == -1 or horiz == -1
            return vert + 1 + 100 * (horiz + 1)


        ans = 0
        for grid in patterns:
            summ = summary(grid)
            ans += summ

        print(f"Réponse jour 13 partie 1 : {ans}")

    def partie2():
        with open(filename) as fin:
            patterns = [[list(line) for line in lines.split("\n")]
                        for lines in fin.read().strip().split("\n\n")]


        def is_horiz(grid, i):
            n, m = len(grid), len(grid[0])
            # Vertical line of reflection
            for j in range(m):
                for k1 in range(n):
                    k2 = i*2+1 - k1
                    if not (0 <= k2 < n):
                        continue
                    if grid[k1][j] != grid[k2][j]:
                        return False

            return True


        def transpose(grid):
            return list(zip(*grid))


        def summary(grid, avoid=(-1, -1)):
            n, m = len(grid), len(grid[0])

            horiz = -1
            for i in range(n-1):
                if i != avoid[0] and is_horiz(grid, i):
                    horiz = i
                    break

            vert = -1
            T = transpose(grid)
            for j in range(m-1):
                if j != avoid[1] and is_horiz(T, j):
                    vert = j
                    break

            return (horiz, vert)


        def summary_2(grid):
            """Fix a smudge, recompute summary."""
            n, m = len(grid), len(grid[0])
            summ_og = summary(grid)

            for i in range(n):
                for j in range(m):
                    grid_copy = deepcopy(grid)
                    grid_copy[i][j] = "." if grid[i][j] == "#" else "#"

                    """ print(i, j)
                    print("\n".join(["".join(line) for line in grid_copy])) """

                    summ_new = summary(grid_copy, avoid=summ_og)

                    if summ_new not in [summ_og, (-1, -1)]:
                        if summ_new[0] != -1:
                            contrib = (summ_new[0] + 1) * 100
                        else:
                            assert summ_new[1] != -1
                            contrib = summ_new[1] + 1

                        return contrib


        ans = 0
        for grid in patterns:
            summ = summary_2(grid)
            ans += summ

        print(f"Réponse jour 13 partie 2 : {ans}")
    
    partie1()
    partie2()
    
j13()

Réponse jour 13 partie 1 : 35521
Réponse jour 13 partie 2 : 34795


In [61]:
def j14():
    filename = "Input/inputj14.txt"
        
    def partie1():
        with open(filename) as fin:
            lines = fin.read().strip().split("\n")

        n, m = len(lines), len(lines[0])

        # Tilt all the rocks up


        def process_col(j):
            # Process column j
            i = 0
            ans = 0

            while i < n:
                while i < n and lines[i][j] == "#":
                    i += 1

                # Now we're at an O or a .
                count = 0
                start = i
                while i < n and lines[i][j] != "#":
                    if lines[i][j] == "O":
                        count += 1
                    i += 1

                for ii in range(start, start + count):
                    ans += n - ii

            return ans


        ans = 0
        for j in range(m):
            ans += process_col(j)

        print(f"Réponse jour 14 partie 1 : {ans}")
    
    def partie2():
        with open(filename) as fin:
            lines = [list(line) for line in fin.read().strip().split("\n")]

        n, m = len(lines), len(lines[0])

        # North, west, south, east


        def north_load(grid):
            n, m = len(grid), len(grid[0])

            # Process column j
            ans = 0
            for i in range(n):
                for j in range(m):
                    if grid[i][j] == "O":
                        ans += n - i
            return ans


        def tilt_up(grid):
            n, m = len(grid), len(grid[0])

            # Process column j
            for j in range(m):
                i = 0
                while i < n:
                    while i < n and grid[i][j] == "#":
                        i += 1

                    # Now we're at an O or a .
                    count = 0
                    start = i
                    while i < n and grid[i][j] != "#":
                        if grid[i][j] == "O":
                            count += 1
                        i += 1

                    for ii in range(start, start + count):
                        grid[ii][j] = "O"
                    for ii in range(start + count, i):
                        grid[ii][j] = "."

            return grid


        def rotate_90(grid):
            # Rotate by 90 deg counterclockwise
            new_grid = [[None] * n for _ in range(m)]
            for i in range(m):
                for j in range(n):
                    new_grid[i][j] = grid[j][m-1-i]
            return new_grid


        def rotate(grid, i):
            # Rotate grid by i*90 deg counterclockwise
            grid_copy = deepcopy(grid)
            for _ in range(i % 4):
                grid_copy = rotate_90(grid_copy)
            return grid_copy


        cycle2grid = {}
        seen = {}


        def get_hash(grid):
            return "\n".join(["".join(line) for line in grid])


        def do_cycle(grid):
            grid_copy = deepcopy(grid)
            for i in range(4):
                grid_copy = rotate(grid_copy, 4 - (i % 4))
                grid_copy = tilt_up(grid_copy)
                grid_copy = rotate(grid_copy, i % 4)
            return grid_copy


        grid = lines
        for cycle in range(10**9):
            grid = do_cycle(grid)

            h = get_hash(grid)
            if h in seen:
                period = cycle - seen[h]
                ans_grid = cycle2grid[(10**9 - 1 - seen[h]) % period + seen[h]]
                break

            seen[get_hash(grid)] = cycle
            cycle2grid[cycle] = deepcopy(grid)
        print(f"Réponse jour 14 partie 2 : {north_load(ans_grid)}")
    
    partie1()
    partie2()
    
j14()

Réponse jour 14 partie 1 : 105461
Réponse jour 14 partie 2 : 102829


In [63]:
def j15():
    filename = "Input/inputj15.txt"
        
    def partie1():
        with open(filename) as fin:
            line = fin.read().strip()


        def HASH(s):
            cur = 0
            for c in s:
                cur += ord(c)
                cur *= 17
                cur %= 256
            return cur


        parts = line.split(",")
        cur = 0
        ans = 0
        for part in parts:
            cur = HASH(part)
            ans += cur

        print(f"Réponse jour 15 partie 1 : {ans}")
    
    def partie2():
        with open(filename) as fin:
            line = fin.read().strip()


        def HASH(s):
            cur = 0
            for c in s:
                cur += ord(c)
                cur *= 17
                cur %= 256
            return cur


        boxes = [[] for _ in range(256)]

        parts = line.split(",")
        for part in parts:
            if "-" in part:
                label = part[:part.index("-")]
                box = HASH(label)
                lens = list(filter(lambda x: x[0] == label, boxes[box]))
                if len(lens) > 0:
                    idx = boxes[box].index(lens[0])
                    boxes[box].pop(idx)

            if "=" in part:
                label = part[:part.index("=")]
                box = HASH(label)
                focal_len = int(part[part.index("=")+1:])

                lens = list(filter(lambda x: x[0] == label, boxes[box]))
                if len(lens) > 0:
                    idx = boxes[box].index(lens[0])
                    boxes[box][idx] = [label, focal_len]
                else:
                    boxes[box].append([label, focal_len])

        ans = 0

        for i, box in enumerate(boxes):
            power = 0
            for j, lens in enumerate(box):
                power += (1 + i) * (j + 1) * lens[1]

            ans += power

        print(f"Réponse jour 15 partie 2 : {ans}")
    
    partie1()
    partie2()
    
j15()

Réponse jour 15 partie 1 : 516070
Réponse jour 15 partie 2 : 244981


In [68]:
def j16():
    filename = "Input/inputj16.txt"
        
    def partie1():
        with open(filename) as fin:
            lines = fin.read().strip().split("\n")

        """
        Direction key:
        1
        2 . 0
        3
        """

        N = len(lines)
        M = len(lines[0])
        ins = [[[False] * 4 for _ in range(M)] for _ in range(N)]

        d_pos = [(0, 1), (-1, 0), (0, -1), (1, 0)]


        def enter(i, j, d):
            """
            Enter tile @ row i, col j, direction d
            """
            if not (0 <= i < N and 0 <= j < M):
                return
            if ins[i][j][d]:
                return
            ins[i][j][d] = True

            tile = lines[i][j]

            # Pass through
            if tile == ".":
                out_dirs = [d]
            # Splitters
            # Use fancy XOR to cut down on redundancy
            elif tile in ["-", "|"]:
                if (d in [1, 3]) ^ (tile == "-"):
                    out_dirs = [d]
                else:
                    out_dirs = [(d + 1) % 4, (d + 3) % 4]
            # Mirrors
            elif tile == "/":
                out_dirs = [[1, 0, 3, 2][d]]
            elif tile == "\\":
                out_dirs = [[3, 2, 1, 0][d]]

            for out_d in out_dirs:
                ii = i + d_pos[out_d][0]
                jj = j + d_pos[out_d][1]
                enter(ii, jj, out_d)


        enter(0, 0, 0)
        count = 0
        for i in range(N):
            for j in range(M):
                if any(ins[i][j]):
                    count += 1

        print(f"Réponse jour 16 partie 1 : {count}")
    
    def partie2():
        with open(filename) as fin:
            lines = fin.read().strip().split("\n")

        """
        Direction key:
        1
        2 . 0
        3
        """

        N = len(lines)
        M = len(lines[0])

        d_pos = [(0, 1), (-1, 0), (0, -1), (1, 0)]


        def enter(i, j, d):
            """
            Enter tile @ row i, col j, direction d
            """
            if not (0 <= i < N and 0 <= j < M):
                return
            if ins[i][j][d]:
                return
            ins[i][j][d] = True

            tile = lines[i][j]

            # Pass through
            if tile == ".":
                out_dirs = [d]
            # Splitters
            # Use fancy XOR to cut down on redundancy
            elif tile in ["-", "|"]:
                if (d in [1, 3]) ^ (tile == "-"):
                    out_dirs = [d]
                else:
                    out_dirs = [(d + 1) % 4, (d + 3) % 4]
            # Mirrors
            elif tile == "/":
                out_dirs = [[1, 0, 3, 2][d]]
            elif tile == "\\":
                out_dirs = [[3, 2, 1, 0][d]]

            for out_d in out_dirs:
                ii = i + d_pos[out_d][0]
                jj = j + d_pos[out_d][1]
                enter(ii, jj, out_d)


        def energy(i, j, d):
            """
            Max energy if enter from tile (i, j) with direction d
            """
            global ins

            count = 0
            ins = [[[False] * 4 for _ in range(M)] for _ in range(N)]
            enter(i, j, d)
            for i in range(N):
                for j in range(M):
                    count += any(ins[i][j])

            return count


        best = 0
        for j in range(M):
            best = max(best, energy(0, j, 3))
        for j in range(M):
            best = max(best, energy(N - 1, j, 1))
        for i in range(N):
            best = max(best, energy(i, 0, 0))
        for i in range(N):
            best = max(best, energy(i, M - 1, 2))

        print(f"Réponse jour 16 partie 2 : {best}")
    
    partie1()
    partie2()
    
j16()

Réponse jour 16 partie 1 : 7951
Réponse jour 16 partie 2 : 8148


In [71]:
def j17():
    filename = "Input/inputj17.txt"
    
    def get_min(grid, min_l=1, max_l=3):
        s = (0, 0)
        e = (len(grid[0]) - 1, len(grid) - 1)

        max_x = len(grid[0])
        max_y = len(grid)

        dirs = {">": (1, 0), "v": (0, 1), "^": (0, -1), "<": (-1, 0)}
        opposite = {"<": ">", ">": "<", "v": "^", "^": "v"}

        to_visit = [(0, ">", s), (0, "v", s)]

        seen = set()

        while to_visit:
            cl, cd, cp = heapq.heappop(to_visit)
            if (cp, cd) in seen:
                continue

            seen.add((cp, cd))
            path_len = len(cd)

            for d in dirs:
                n_p = (cp[0] + dirs[d][0], cp[1] + dirs[d][1])
                if (
                    not 0 <= n_p[0] < max_x  # out of grid range
                    or not 0 <= n_p[1] < max_y
                    or (d == cd[-1] and path_len == max_l)  # path longer
                    or (d != cd[-1] and path_len < min_l)  # path shorter
                    or cd[-1] == opposite[d]  # turning back
                ):
                    continue
                if d == cd[-1]:
                    nd = cd + d
                else:
                    nd = d
                if (n_p, nd) in seen:
                    continue
                if n_p == e and len(nd) >= min_l:
                    return cl + grid[n_p[1]][n_p[0]]
                heapq.heappush(to_visit, (cl + grid[n_p[1]][n_p[0]], nd, n_p))
        
    def partie1():
        grid = [list(map(int, l.strip())) for l in open(filename)]

        print(f"Réponse jour 17 partie 1 : {get_min(grid)}")
    
    def partie2():
        grid = [list(map(int, l.strip())) for l in open(filename)]

        print(f"Réponse jour 17 partie 2 : {get_min(grid, 4, 10)}")
    
    partie1()
    partie2()
    
j17()

Réponse jour 17 partie 1 : 791
Réponse jour 17 partie 2 : 900


In [1]:
def j18():
    filename = "Input/inputj18.txt"
    
    def shoe_lace(points):
        res = sum(points[i - 1][0] * points[i][1] - points[i - 1][1] * points[i][0] for i in range(len(points)))
        return res // 2
        
    def partie1():
        polygon = [(0, 0)]

        deltas = {"R": (1, 0), "L": (-1, 0), "U": (0, -1), "D": (0, 1)}

        p_cnt = 0

        for l in open(filename):
            ps = l.split()
            d = deltas[ps[0]]
            lp = polygon[-1]
            steps = int(ps[1])
            p_cnt += steps
            polygon.append((lp[0] + d[0] * steps, lp[1] + d[1] * steps))

        print(f"Réponse jour 18 partie 1 : {shoe_lace(polygon) + p_cnt // 2 + 1}")
    
    def partie2():
        polygon = [(0, 0)]

        deltas = {"0": (1, 0), "2": (-1, 0), "3": (0, -1), "1": (0, 1)}

        p_cnt = 0

        for l in open(filename):
            ps = l.split()

            d = deltas[ps[2][-2]]
            lp = polygon[-1]
            steps = int(ps[2][2:-2], 16)

            p_cnt += steps
            polygon.append((lp[0] + d[0] * steps, lp[1] + d[1] * steps))

        print(f"Réponse jour 18 partie 2 : {shoe_lace(polygon) + (p_cnt + 1) // 2 + 1}")
    
    partie1()
    partie2()
    
j18()

Réponse jour 18 partie 1 : 106459
Réponse jour 18 partie 2 : 63806916814808


In [5]:
def j19():
    filename = "Input/inputj19.txt"
    
    def get_combs(state, rules, ranges={"x": (1, 4000), "m": (1, 4000), "a": (1, 4000), "s": (1, 4000)}):
        if state == "R":
            return 0
        elif state == "A":
            x_s = ranges["x"]
            m_s = ranges["m"]
            a_s = ranges["a"]
            s_s = ranges["s"]

            return (x_s[1] - x_s[0] + 1) * (m_s[1] - m_s[0] + 1) * (a_s[1] - a_s[0] + 1) * (s_s[1] - s_s[0] + 1)

        total = 0
        for pred in rules[state].split(","):
            if not ":" in pred:
                total += get_combs(pred, rules, ranges)
            else:
                p = pred.split(":")
                new_range = deepcopy(ranges)

                new_val = int(p[0][2:])
                c_range = ranges[p[0][0]]

                if c_range[0] < new_val < c_range[1]:
                    if p[0][1] == "<":
                        new_range[p[0][0]] = (c_range[0], new_val - 1)
                        ranges[p[0][0]] = (new_val, c_range[1])
                    else:
                        new_range[p[0][0]] = (new_val + 1, c_range[1])
                        ranges[p[0][0]] = (c_range[0], new_val)
                    total += get_combs(p[1], rules, new_range)
        return total
        
    def partie1():
        ps = open(filename).read().split("\n\n")

        rules = {}

        for l in ps[0].splitlines():
            p = l.split("{")
            rules[p[0]] = p[1][:-1]

        A = 0
        for p in ps[1].splitlines():
            state = "in"
            vals = {}
            for c in p[1:-1].split(","):
                vals[c.split("=")[0]] = int(c.split("=")[1])
            while state not in "AR":
                current_cond = rules[state].split(",")
                for pred in current_cond:
                    if ":" in pred:
                        evs = pred.split(":")
                        if eval('vals["' + evs[0][0] + '"]' + "".join(evs[0][1:])):
                            state = evs[1]
                            break
                    else:
                        state = pred

                if state == "A":
                    A += sum(vals.values())
                    break
                elif state == "R":
                    break

        print(f"Réponse jour 19 partie 1 : {A}")
    
    def partie2():
        ps = open(filename).read().split("\n\n")

        rules = {}

        for l in ps[0].splitlines():
            p = l.split("{")
            rules[p[0]] = p[1][:-1]

        print(f"Réponse jour 19 partie 2 : {get_combs('in', rules)}")
    
    partie1()
    partie2()
    
j19()

Réponse jour 19 partie 1 : 368964
Réponse jour 19 partie 2 : 127675188176682


In [9]:
def j20():
    filename = "Input/inputj20.txt"

    class Module:
        def __init__(self, l) -> None:
            p = l.split(" -> ")
            self.name = p[0]
            if p[0][0] in "%&":
                self.type = p[0][0]
                self.name = "".join(p[0][1:])
            else:
                self.type = ""
                self.name = p[0]
            self.next = p[1].split(", ")
            self.state = False
        
    def partie1():
        modules = {}
        for l in open(filename).read().splitlines():
            m = Module(l)

            modules[m.name] = m

        for m in modules:
            if modules[m].type == "&":
                modules[m].input = {}
                for d in modules:
                    if m in modules[d].next:
                        modules[m].input[d] = False

        high = 0
        lows = 0

        for _ in range(1000):
            to_send = [("broadcaster", False)]

            while to_send:
                src, sig = to_send.pop(0)

                dst = []
                output = sig

                if sig:
                    high += 1
                else:
                    lows += 1


                if src in modules and modules[src].type == "":
                    dst = modules[src].next
                elif src in modules and modules[src].type == "%" and not sig:
                    output = modules[src].state = not modules[src].state
                    dst = modules[src].next
                elif src in modules and modules[src].type == "&":
                    output = not all(modules[src].input.values())
                    dst = modules[src].next

                for m in dst:
                    to_send.append((m, output))
                    if m in modules and modules[m].type == "&":
                        modules[m].input[src] = output

        print(f"Réponse jour 20 partie 1 : {high * lows}")
    
    def partie2():
        modules = {}
        for l in open(filename).read().splitlines():
            m = Module(l)

            modules[m.name] = m
            if "rx" in m.next:
                main_module = m.name

        for m in modules:
            if modules[m].type == "&":
                modules[m].input = {}
                for d in modules:
                    if m in modules[d].next:
                        modules[m].input[d] = False

        cycles = {m:0 for m in modules[main_module].input}

        cycle_cnt = 0
        while not all(cycles.values()):
            cycle_cnt += 1
            to_send = [("broadcaster", False)]

            while to_send:
                src, sig = to_send.pop(0)

                dst = []
                output = sig


                if src in modules and modules[src].type == "":
                    dst = modules[src].next
                elif src in modules and modules[src].type == "%" and not sig:
                    output = modules[src].state = not modules[src].state
                    dst = modules[src].next
                elif src in modules and modules[src].type == "&":
                    output = not all(modules[src].input.values())
                    dst = modules[src].next

                    if src == main_module and any(modules[src].input.values()):
                        for m in modules[src].input:
                            if modules[src].input[m]:
                                cycles[m] = cycle_cnt

                for m in dst:
                    to_send.append((m, output))
                    if m in modules and modules[m].type == "&":
                        modules[m].input[src] = output

        print(f"Réponse jour 20 partie 2 : {prod(cycles.values())}")
    
    partie1()
    partie2()
    
j20()

Réponse jour 20 partie 1 : 834323022
Réponse jour 20 partie 2 : 225386464601017


In [15]:
def j21():
    filename = "Input/inputj21.txt"
        
    def partie1():
        grid = set()

        for y, l in enumerate(open(filename)):
            for x, c in enumerate(l):
                if c == "#":
                    grid.add(x + y * 1j)
                elif c == "S":
                    start = x + y * 1j

        deltas = [1, -1, -1j, 1j]

        reach = {0: set([start])}

        while len(reach) <= 64:
            steps = max(reach.keys())

            reach[steps + 1] = set()

            for pos in reach[steps]:
                for d in deltas:
                    if pos + d not in grid:
                        reach[steps + 1].add(pos + d)

        print(f"Réponse jour 21 partie 1 : {len(reach[64])}")
    
    def quadratic(P, n):
        a = (P[2] - 2 * P[1] + P[0]) / 2
        b = P[1] - P[0] - a
        c = P[0]
        return a * (n * n) + b * n + c

    def partie2():
        D = [i.strip() for i in open(filename, "r").readlines()]
        g, w, h, start = grid_from_strs(D, find="S")

        goal, s = 26501365, 26501365 % w
        p2stop = (s + 2 * w) + 1 # 328

        def compute_reachable():
            reachable = defaultdict(int)
            Q, visited = deque([(start, 0)]), defaultdict(set)
            visited[0] = set([start])
            while Q:
                p, steps = Q.popleft()
                if steps == p2stop:
                    break
                reachable[steps] += 1
                for d in DIR:
                    q = (p[0] + d[0], p[1] + d[1])
                    if g[(p[1]+d[1]) % h][(p[0]+d[0]) % w] != "#":
                        if not steps+1 in reachable:
                            reachable[steps+1] = reachable[steps-1]
                            visited[steps+1] = visited[steps-1]
                        if q not in visited[steps+1]:
                            Q.append((q, steps + 1))
                            visited[steps+1].add(q)
            return reachable
        reachable = compute_reachable()
        P = reachable[s], reachable[s + w], reachable[s + 2 * w]

        print(f"Réponse jour 21 partie 2 : {int(quadratic(P, goal // w))}")
    
    partie1()
    partie2()
    
j21()

Réponse jour 21 partie 1 : 3585
Réponse jour 21 partie 2 : 597102953699891


In [17]:
def j22():
    filename = "Input/inputj22.txt"
    
    def drop(bricks, eraly=False):
        dropped = set()
        seen = set()

        for i in range(len(bricks)):
            pts = bricks[i]

            while True:
                new_pts = {(p[0], p[1], p[2] - 1) for p in pts}
                if all(p not in seen and p[2] > 0 for p in new_pts):
                    bricks[i] = new_pts
                    pts = new_pts
                    dropped.add(i)
                    if eraly:
                        return 1
                else:
                    break

            seen.update(pts)

        return len(dropped)
        
    def partie1():
        bricks = []

        for l in open(filename):
            p = list(map(lambda x: tuple(map(int, x.split(","))), l.strip().split("~")))
            b = set([p[0], p[1]])

            idx_deltas = [c1 != c2 for c1, c2 in zip(p[0], p[1])]
            if any(idx_deltas):
                assert sum(idx_deltas) == 1
                idx = idx_deltas.index(True)
                for val in range(min(p[0][idx], p[1][idx]), max(p[0][idx], p[1][idx])):
                    b.add(tuple(val if i == idx else c for i, c in enumerate(p[0])))
            bricks.append(b)

        bricks.sort(key=lambda x: min(p[2] for p in x))

        drop(bricks)
        total = 0

        for i in range(len(bricks)):
            new_bricks = bricks.copy()
            del new_bricks[i]
            total += not drop(new_bricks, True)

        print(f"Réponse jour 22 partie 1 : {total}")
    
    def partie2():
        bricks = []

        for l in open(filename):
            p = list(map(lambda x: tuple(map(int, x.split(","))), l.strip().split("~")))
            b = set([p[0], p[1]])

            idx_deltas = [c1 != c2 for c1, c2 in zip(p[0], p[1])]
            if any(idx_deltas):
                assert sum(idx_deltas) == 1
                idx = idx_deltas.index(True)
                for val in range(min(p[0][idx], p[1][idx]), max(p[0][idx], p[1][idx])):
                    b.add(tuple(val if i == idx else c for i, c in enumerate(p[0])))
            bricks.append(b)

        bricks.sort(key=lambda x: min(p[2] for p in x))

        drop(bricks)
        total = 0

        for i in range(len(bricks)):
            new_bricks = bricks.copy()
            del new_bricks[i]
            total += drop(new_bricks)
            
        print(f"Réponse jour 22 partie 2 : {total}")
    
    partie1()
    partie2()
    
j22()

Réponse jour 22 partie 1 : 409
Réponse jour 22 partie 2 : 61097


In [22]:
def j23():
    filename = "Input/inputj23.txt"
    
    D = [i.strip() for i in open(filename, "r").readlines()]
    tot = 0

    global g, w, h, visited, mm, mid_points
    g = [[ch for ch in row] for row in D]
    h = len(g)
    w = len(g[0])
    start = (1, 0)
    end = (w - 2, h - 1)
    
    def grid_neighbors(p, part1=False):
        width = w
        height = h
        r = []
        dd = None
        if part1:
            if g[p[1]][p[0]] == ">":
                dd = (1, 0)
            elif g[p[1]][p[0]] == "<":
                dd = (-1, 0)
            elif g[p[1]][p[0]] == "^":
                dd = (0, -1)
            elif g[p[1]][p[0]] == "v":
                dd = (0, 1)
        for d in [dd] if dd else [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            q = (p[0] + d[0], p[1] + d[1])
            if (
                q[0] < 0
                or q[1] < 0
                or q[0] >= width
                or q[1] >= height
                or g[q[1]][q[0]] == "#"
            ):
                continue
            r.append(q)
        return r
    
    def find_all_intersections(part1=False):
        res = []
        for x in range(w):
            for y in range(h):
                if g[y][x] == "." and len(grid_neighbors((x, y), part1)) > 2:
                    res.append((x, y))
        return res


    def find_edges_from_isect_to_isect(isects, start, end, part1=False):
        graph = {}
        for i in isects:
            o, p = i, i
            dests = []
            for d in grid_neighbors(o, part1):
                vv, l = set(), 1
                p = d
                vv.add(o)
                vv.add(p)
                found = True
                while p not in isects:
                    nxt = [v for v in grid_neighbors(p, part1) if v not in vv]
                    if len(nxt) == 0:
                        found = False
                        break
                    l += 1
                    p = nxt[0]
                    vv.add(p)
                if found:
                    dests.append((p[0], p[1], l))  # graph is origin -> (dest, len)
            graph[(o[0],o[1])] = dests
        # Remap the whole graph to integer node names so we
        # can mark visited really quickly.
        gg = {}
        map_ = {}
        for i,k in enumerate(graph.keys()):
            map_[k] = i
        for k,v in graph.items():
            gg[map_[k]] = [(map_[(x[0],x[1])],x[2]) for x in v]
        return gg,map_[start],map_[end]

    ans = []
    for part1 in [True, False]:
        isect = set(find_all_intersections(part1)) | {start, end}
        graph,start_,end_ = find_edges_from_isect_to_isect(isect, start, end, part1)
        
        mid_points = defaultdict(list)

        MID_DEPTH = int(len(graph.keys()) / 2) - 2
        mm = 0
        visited = 0

        def outbound_dfs(n, l, d):
            global mm, visited, mid_points
            if visited & 1<<n:
                return
            if not part1 and d == MID_DEPTH:
                mid_points[n].append((l,visited))
                return
            visited |= 1<<n
            if n == end_:
                mm = max(l, mm)
            else:
                for nn, ll in graph[n]:
                    outbound_dfs(nn, l + ll, d+1)
            visited ^= 1<<n


        def return_dfs(n, l, d):
            global mm, visited, mid_points
            if d == MID_DEPTH:
                return
            if visited & 1<<n:
                return
            visited |= 1<<n
            if n == start_:
                mm = max(l, mm)
            else:
                for nn, ll in graph[n]:
                    if nn in mid_points:
                        for l_, visited_ in mid_points[nn]:
                            if l+ll+l_ < mm:
                                break
                            if (visited_|1<<nn) & visited == 0:
                                mm = max(l+ll+l_, mm)
                    return_dfs(nn, l + ll, d+1)
            visited ^= 1<<n

        sadd, eadd = 0,0
        start_, sadd = graph[start_][0]
        if not part1:
            end_, eadd = graph[end_][0]

        outbound_dfs(start_, sadd, 0)
        if not part1:
            for k in mid_points.keys():
                mid_points[k] = sorted(mid_points[k], reverse=True)
            return_dfs(end_, eadd, 0)
        
        ans.append(mm)
        
    def partie1():
        print(f"Réponse jour 23 partie 1 : {ans[0]}")
    
    def partie2():
        print(f"Réponse jour 23 partie 2 : {ans[1]}")
    
    partie1()
    partie2()
    
j23()

Réponse jour 23 partie 1 : 2178
Réponse jour 23 partie 2 : 6486


In [25]:
def j24():
    filename = "Input/inputj24.txt"
    
    def get_coef(p, v):
        a = v[1] / v[0]
        b = p[1] - (p[0] * v[1]) / v[0]

        return a, b


    def get_intrestion(coef1, coef2):
        if coef2[0] != coef1[0]:
            return (coef2[1] - coef1[1]) / (coef1[0] - coef2[0])
        return -1


    def get_time(d, d0, v):
        return (d - d0) / v
        
    def partie1():
        particles = []

        for l in open(filename):
            p = l.split(" @ ")
            ps = list(map(int, p[0].split(", ")))
            vs = list(map(int, p[1].split(", ")))

            particles.append((ps, vs))

        total = 0

        lower_limit = 200000000000000
        upper_limit = 400000000000000

        for p1, p2 in combinations(particles, 2):
            pos1, v1 = p1
            pos2, v2 = p2

            c1 = get_coef(pos1, v1)
            c2 = get_coef(pos2, v2)

            x = get_intrestion(c1, c2)
            y = c1[0] * x + c1[1]

            tx1 = get_time(x, pos1[0], v1[0])
            tx2 = get_time(x, pos2[0], v2[0])

            if tx1 >= 0 and tx2 >= 0 and lower_limit <= x <= upper_limit and lower_limit <= y <= upper_limit:
                total += 1
        print(f"Réponse jour 24 partie 1 : {total}")
    
    def partie2():
        particles = []

        sx = var("sx")
        sy = var("sy")
        sz = var("sz")

        vx = var("vx")
        vy = var("vy")
        vz = var("vz")

        eq = []

        for l in open(filename):
            p = l.split(" @ ")
            ps = list(map(int, p[0].split(", ")))
            vs = list(map(int, p[1].split(", ")))

            particles.append((ps, vs))

            ts = "t{}".format(len(eq) // 3)
            exec(f'{ts} = var("{ts}")')

            eq.append(Eq(eval(f"sx + vx * {ts}"), eval(f"ps[0] + vs[0] * {ts}")))
            eq.append(Eq(eval(f"sy + vy * {ts}"), eval(f"ps[1] + vs[1] * {ts}")))
            eq.append(Eq(eval(f"sz + vz * {ts}"), eval(f"ps[2] + vs[2] * {ts}")))

            # 3 pos, 3 vs + 3 ts minimum length, otherwise same solution with more calcaution
            if len(eq) > 9:
                break

        ans = solve(eq)[0]
        print(f"Réponse jour 24 partie 2 : {ans[sx] + ans[sy] + ans[sz]}")
    
    partie1()
    partie2()
    
j24()

Réponse jour 24 partie 1 : 24192
Réponse jour 24 partie 2 : 664822352550558


In [36]:
def j25():
    filename = "Input/inputj25.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        graph = networkx.Graph()

        for l in open(filename):
            src, dst = l.strip().split(": ")

            for n in dst.split():
                graph.add_edge(src, n)

        _, partitions = networkx.stoer_wagner(graph)

        print(f"Réponse jour 25 partie 1 : {len(partitions[0]) * len(partitions[1])}")
    
    def partie2():
        print("Réponse jour 25 partie 2 : sur le site de aoc")
    
    partie1()
    partie2()
    
j25()

Réponse jour 25 partie 1 : 603368
Réponse jour 25 partie 2 : sur le site de aoc
